# Load Dependencies

In [7]:
# Import necessary libraries for loading and processing data
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

# Import Keras for implementing autoencoders
import keras

# Load and Preprocess Data

In [8]:
# load the Iris dataset
iris = load_iris()
data = iris.data
target = iris.target

# Standardize the data
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                    test_size=0.2, random_state=42)

# Define Autoencoder Architecture

In [9]:
# Define the autoencoder architecture
input_dim = X_train.shape[1]
encoding_dim = 2  # Set the encoding dimension

input_layer = keras.layers.Input(shape=(input_dim,))
encoder = keras.layers.Dense(encoding_dim, activation="relu")(input_layer)
decoder = keras.layers.Dense(input_dim, activation="sigmoid")(encoder)

autoencoder = keras.Model(inputs=input_layer, outputs=decoder)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

# Summary of the autoencoder architecture
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 10        
                                                                 
 dense_3 (Dense)             (None, 4)                 12        
                                                                 
Total params: 22
Trainable params: 22
Non-trainable params: 0
_________________________________________________________________


# Train Autoencoder

In [10]:
# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=100, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

Epoch 1/100
4/4 [==============================] - 1s 65ms/step - loss: 1.2621 - val_loss: 1.1534
Epoch 2/100
4/4 [==============================] - 0s 22ms/step - loss: 1.2584 - val_loss: 1.1499
Epoch 3/100
4/4 [==============================] - 0s 20ms/step - loss: 1.2548 - val_loss: 1.1461
Epoch 4/100
4/4 [==============================] - 0s 16ms/step - loss: 1.2512 - val_loss: 1.1421
Epoch 5/100
4/4 [==============================] - 0s 19ms/step - loss: 1.2475 - val_loss: 1.1381
Epoch 6/100
4/4 [==============================] - 0s 23ms/step - loss: 1.2433 - val_loss: 1.1341
Epoch 7/100
4/4 [==============================] - 0s 16ms/step - loss: 1.2395 - val_loss: 1.1300
Epoch 8/100
4/4 [==============================] - 0s 18ms/step - loss: 1.2355 - val_loss: 1.1258
Epoch 9/100
4/4 [==============================] - 0s 15ms/step - loss: 1.2313 - val_loss: 1.1217
Epoch 10/100
4/4 [==============================] - 0s 16ms/step - loss: 1.2274 - val_loss: 1.1175
Epoch 11/100
4/4 [=

Epoch 84/100
4/4 [==============================] - 0s 26ms/step - loss: 0.9295 - val_loss: 0.8174
Epoch 85/100
4/4 [==============================] - 0s 23ms/step - loss: 0.9264 - val_loss: 0.8139
Epoch 86/100
4/4 [==============================] - 0s 23ms/step - loss: 0.9232 - val_loss: 0.8104
Epoch 87/100
4/4 [==============================] - 0s 23ms/step - loss: 0.9201 - val_loss: 0.8070
Epoch 88/100
4/4 [==============================] - 0s 22ms/step - loss: 0.9171 - val_loss: 0.8036
Epoch 89/100
4/4 [==============================] - 0s 15ms/step - loss: 0.9140 - val_loss: 0.8004
Epoch 90/100
4/4 [==============================] - 0s 17ms/step - loss: 0.9111 - val_loss: 0.7971
Epoch 91/100
4/4 [==============================] - 0s 17ms/step - loss: 0.9081 - val_loss: 0.7939
Epoch 92/100
4/4 [==============================] - 0s 24ms/step - loss: 0.9053 - val_loss: 0.7907
Epoch 93/100
4/4 [==============================] - 0s 16ms/step - loss: 0.9023 - val_loss: 0.7877
Epoch 94/1

# Select Best Features with Autoencoder

In [11]:
# Extract features from the encoder part of the autoencoder
encoder = keras.Model(inputs=autoencoder.input, outputs=autoencoder.layers[1].output)
encoded_features_train = encoder.predict(X_train)
encoded_features_test = encoder.predict(X_test)

# Display the shape of extracted features
print("Encoded Features Shape (Train):", encoded_features_train.shape)
print("Encoded Features Shape (Test):", encoded_features_test.shape)

1/1 [==============================] - 0s 24ms/step
Encoded Features Shape (Train): (120, 2)
Encoded Features Shape (Test): (30, 2)


# Use Selected Features for Training

In [13]:
# Import a predictive model (e.g., logistic regression)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Fit a logistic regression model using the selected features
model = LogisticRegression()
model.fit(encoded_features_train, y_train)

# Make predictions on the test set
y_pred = model.predict(encoded_features_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy with Selected Features:", accuracy)

Accuracy with Selected Features: 1.0
